Basic RNN Model

In [1]:
import tensorflow as tf
import numpy as np

# reset graph to make sure we keep same random numbers same
def reset_graph(seed=2018):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
reset_graph()
X0_data = np.array([[1, 2],[3, 4],[5, 6]])
X1_data = np.array([[7, 8],[9, 10],[11, 12]])
X2_data = np.array([[13, 14],[15, 16],[17, 18]])

In [4]:
# hyperparams
n_neurons = 8

# parameters
n_inputs = 2

# build your Seq2Seq RNN model
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
X2 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal([n_inputs, n_neurons]))
b  = tf.Variable(tf.zeros([1, n_neurons]))
Wy = tf.Variable(tf.random_normal([n_neurons, n_neurons]))

y0 = tf.tanh(tf.matmul(X0, Wx) + b) 
y1 = tf.tanh(tf.matmul(y0, Wy) + tf.matmul(X1, Wx) + b)
y2 = tf.tanh(tf.matmul(y1, Wy) + tf.matmul(X2, Wx) + b)

output0 = tf.layers.dense(y0, 1)
output1 = tf.layers.dense(y1, 1)
output2 = tf.layers.dense(y2, 1)

In [6]:
# initialize variables
init = tf.global_variables_initializer()

# train
with tf.Session() as sess:
    sess.run(init)
    output0_eval = sess.run(output0, feed_dict={X0:X0_data,X1:X1_data, X2:X2_data})
    output1_eval = sess.run(output1, feed_dict={X0:X0_data,X1:X1_data, X2:X2_data})
    output2_eval = sess.run(output2, feed_dict={X0:X0_data,X1:X1_data, X2:X2_data})
    print('input0: {} input1:{} input2: {} -> output0: {} output1: {} output2: {}'.format(
    X0_data[0], X1_data[0], X2_data[0], output0_eval[0], output1_eval[0], output2_eval[0] ))
    print('input0: {} input1:{} input2: {} -> output0: {} output1: {} output2: {}'.format(
    X0_data[1], X1_data[1], X2_data[1], output0_eval[1], output1_eval[1], output2_eval[1] ))
    print('input0: {} input1:{} input2: {} -> output0: {} output1: {} output2: {}'.format(
    X0_data[2], X1_data[2], X2_data[2], output0_eval[2], output1_eval[2], output2_eval[2] ))

input0: [1 2] input1:[7 8] input2: [13 14] -> output0: [-0.32207465] output1: [-0.51884687] output2: [-1.176264]
input0: [3 4] input1:[ 9 10] input2: [15 16] -> output0: [-1.0767608] output1: [-0.48756164] output2: [-1.1769333]
input0: [5 6] input1:[11 12] input2: [17 18] -> output0: [-1.1276517] output1: [-0.48506528] output2: [-1.1774937]


In [8]:
# RNN Basic Cell with static_rnn
reset_graph()

# input data
X_data = np.array([
# 1st.      2nd.     3rd
   [[1, 2],[7, 8],[13, 14]], # first batch
   [[3, 4],[9, 10],[15, 16]],
   [[5, 6],[11, 12],[17, 18]]
])

In [9]:
# hyperparams
n_neurons = 8

# parameters
n_inputs = X_data.shape[2]
n_steps  = X_data.shape[1]

# rnn model
X     = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seq = tf.unstack(X, axis=1)
i     = n_steps

cell          = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output, state = tf.nn.static_rnn(cell, X_seq, dtype=tf.float32)
output_st     = tf.stack(output, axis=1)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [11]:
# initalize 
init = tf.global_variables_initializer()

# train
with tf.Session() as sess:
    sess.run(init)
    feed_dict = {X:X_data}
    X_seq_shape  = sess.run(tf.shape(X_seq), feed_dict=feed_dict)
    output_shape = sess.run(tf.shape(output), feed_dict=feed_dict)
    state_shape  = sess.run(tf.shape(state), feed_dict=feed_dict)
    output_st_shape = sess.run(tf.shape(output_st),feed_dict=feed_dict)
    # print these shapes
    print('X_seq shape [batch_size, n_steps, n_inputs]: ', X_seq_shape)
    print('output shape [batch_size, n_neurons]: ', output_shape)
    print('state shape [batch_size, n_neurons]: ', state_shape)
    print('output_st shape [batch_size, n_steps, n_neurons]: ', output_st_shape)
    
    #evaluate
    output_eval, state_eval = sess.run([output, state], feed_dict=feed_dict)
    print('Is the output of X same as the state? ', np.array_equal(output_eval[2], 
                                                                   state_eval))

X_seq shape [batch_size, n_steps, n_inputs]:  [3 3 2]
output shape [batch_size, n_neurons]:  [3 3 8]
state shape [batch_size, n_neurons]:  [3 8]
output_st shape [batch_size, n_steps, n_neurons]:  [3 3 8]
Is the output of X same as the state?  True
